1. Configuração e Carregamento

* O primeiro passo é carregar as bibliotecas e os dois dataframes de que precisamos:

* df_turismo: Para saber o ranking de popularidade.

* df_interactions: Para saber o "ground truth" (o que os utilizadores gostaram).

In [85]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Definir K (Numero de recomendações que queremos dar)

K = 5

# Carregar o dataset
data_turismo = "../data/turismo_angola_completo.csv"
data_interacoes = "../data/mock_user_interactions.csv"

# Carregar os dados
try:
    df_turismo = pd.read_csv(data_turismo)
    df_interacoes = pd.read_csv(data_interacoes)
    print("Dados carregados com sucesso.")  
except FileNotFoundError:
    print(f"ERRO: Verifique os caminhos dos ficheiros.") 
    print(f"Era esperado: {df_turismo}")
    print(f"Era esperado: {df_interacoes}")   
#print(f"Total de interacções (ratings) no mock é de: {len(df_interacoes)}")
df_turismo.head()

Dados carregados com sucesso.


,Província,Destinos Turísticos,Turistas 2023,Turistas 2024,Receitas (USD),Receitas (AOA)
0,Luanda,"Ilha do Mussulo, Fortaleza de São Miguel, Mira...",125000,137000,72500000,60175000000
1,Benguela,"Praia Morena, Baía Azul, Lobito",89000,97000,45000000,37350000000
2,Huíla,"Serra da Leba, Tundavala, Cristo Rei",65000,73000,36000000,29880000000
3,Namibe,"Deserto do Namibe, Arco do Carvalhão, Lagoa do...",52000,61000,27500000,22825000000
4,Malanje,"Quedas de Kalandula, Pungo Andongo, Parque de ...",41000,49000,21000000,17430000000


2. Construção do Modelo (Ranking de Popularidade)

Este "modelo" é simplesmente uma lista ordenada das províncias, da mais popular para a menos popular, com base nos dados de 2024.

In [80]:
# Ordenar o dataframe de turismo pela coluna "Turismo 204"
df_popularidade = df_turismo.sort_values(by="Turistas 2024", ascending=False) 

# Obter a lista ordenada de provincias mais populares (Nosso Modelo)
racking_popularidade = df_popularidade['Província'].tolist()
print("Ranking de popularidade das províncias (O 'Modelo' Baseline):")
print(racking_popularidade)

# As recomendações do baseline serão as mesmas para todos os utilizadores
recomendacoes_baseline = racking_popularidade[:K]
print(f"\nRecomendações Baseline (Top {K} para TODOS os utilizadores):")
print(recomendacoes_baseline)

Ranking de popularidade das províncias (O 'Modelo' Baseline):
['Luanda', 'Benguela', 'Huíla', 'Namibe', 'Huambo', 'Malanje', 'Cuanza Sul', 'Cabinda', 'Bié', 'Cuanza Norte', 'Uíge', 'Zaire', 'Cuando Cubango', 'Cunene', 'Moxico', 'Lunda Sul', 'Lunda Norte', 'Bengo']

Recomendações Baseline (Top 5 para TODOS os utilizadores):
['Luanda', 'Benguela', 'Huíla', 'Namibe', 'Huambo']


3. Definição do "Ground Truth" (Conjunto de Teste)

Agora, processamos o ficheiro mock_user_interactions.csv. Vamos criar um dicionário onde cada user_id tem uma lista das províncias que realmente gostou (vamos definir "gostou" como rating >= 4).

In [81]:
# Filtrar interações positivas (rating >= 4)
df_gostou = df_interacoes[df_interacoes['rating'] >= 4]

# Criar um dicionário de Ground Truth
# Formato: {user_id: [lista_de_provincias_gostou]}
ground_truth = defaultdict(list)
for _, row in df_gostou.iterrows():
    ground_truth[row['user_id']].append(row['provincia_visitada'])
print("\nGround Truth (O que os utilizadores realmente gostaram):")
# Exemplo para o User 3 (Explorador) e User 1 (Mainstream)
print(f"Usuário 3: {ground_truth[3]}")
print(f"Usuário 1: {ground_truth[1]}") 




Ground Truth (O que os utilizadores realmente gostaram):
Usuário 3: ['Malanje', 'Huambo']
Usuário 1: ['Luanda', 'Benguela']


4. Avaliação Offline (Precisão@k e Recall@k)

Este é o passo central. Vamos iterar por cada utilizador no nosso ground_truth e "fingir" que recomendamos os recomendacoes_baseline. Depois, medimos o quão boas foram essas recomendações.

In [82]:
# Lista para armazenar metricas de avaliação para cada utilizador
lista_precisao_k = []
lista_recall_k = []

# Iterar sobre cada utilizador no nosso conjunto de test (ground truth)
for user_id, provincias_gostou in ground_truth.items():
    # As recomendações do baseline são sempre as mesmas
    recomendacoes_baseline = ['Luanda', 'Benguela', 'Huíla', 'Namibe','Huambo']  # Top 5 províncias mais populares
    # Calcular os Hits (Acertos)
    # Hits são as províncias recomendadas que o utilizador gostou
    hits = [rec for rec in recomendacoes_baseline if rec in provincias_gostou]
    num_hits = len(hits)

    # Calcular Precisão@K
    # De tudo que recomendamos (K), quantos estavam corretos?
    precisao_k = num_hits / K

    # Calcular Recall@K
    # De tudo que o utilizador gostou, quantos é que nós acertámos?
    recall_k = num_hits / len(provincias_gostou)

    lista_precisao_k.append(precisao_k)
    lista_recall_k.append(recall_k) 

# Calculo das medias
media_precisao_k = np.mean(lista_precisao_k)
media_recall_k = np.mean(lista_recall_k)

print(f"--- MÉTRICAS DE AVALIAÇÃO DO MODELO BASELINE ---")
print(f"Média da Precisão@{K}: {media_precisao_k * 100:.2f} %")
print(f"Média do Recall@{K}: {media_recall_k * 100:.2f} %") 

--- MÉTRICAS DE AVALIAÇÃO DO MODELO BASELINE ---
Média da Precisão@5: 26.67 %
Média do Recall@5: 75.00 %


5. Avaliação da Métrica de Negócio (Diversidade)

Agora, vamos calcular a sua métrica de negócio. Qual a percentagem de destinos emergentes que este modelo recomenda?

In [83]:
# Vamos definir "Mainstream" como as Top 3 províncias mais populares
# Podemos ajustar este valor de acordo as nossas necessidades
mainstream = ['Luanda', 'Benguela', 'Huíla']

# Das recomendações feitas, quantas são destinos emergentes?
emergentes_recomendados = [prov for prov in recomendacoes_baseline if prov not in mainstream]
num_emergentes = len(emergentes_recomendados)

# Calcular a percentagem de diversidade nas recomendações
# (numero de destinos emergentes recomendados) / (total de recomendações K)
diversidade_percentagem = (num_emergentes / K) * 100

print(f"--- DIVERSIDADE NAS RECOMENDAÇÕES ---")
print(f"Recomendações feitas (K={K}): {recomendacoes_baseline}")
print(f"Nº Destinos Emergentes Recomendados: {num_emergentes} ({emergentes_recomendados})")
print(f"Percentagem de Diversidade nas Recomendações: {diversidade_percentagem:.2f} %")

--- DIVERSIDADE NAS RECOMENDAÇÕES ---
Recomendações feitas (K=5): ['Luanda', 'Benguela', 'Huíla', 'Namibe', 'Huambo']
Nº Destinos Emergentes Recomendados: 2 (['Namibe', 'Huambo'])
Percentagem de Diversidade nas Recomendações: 40.00 %


## 6. Conclusões e Próximo Passo

O nosso Modelo Baseline (Top-N Popularidade) foi avaliado. Os resultados são:

* **Precisão@5 (Média): 26.67 %**
* **Recall@5 (Média): 75.00 %**
* **Diversidade@5 (Métrica de Negócio): 40.00 %**

**Análise:**

1.  **Prova do Problema:** As métricas de Precisão e Recall são [provavelmente baixas], mostrando que este modelo "tamanho único" falha em satisfazer os utilizadores "Exploradores" (como o User 3, que adorou Malanje e Huambo, e teve 26.67% de precisão).
2.  **Métrica de Negócio:** O modelo [provavelmente recomenda `Huíla`], atingindo uma diversidade de [40.00% se K=5]. Embora este número esteja *acima* da nossa meta de 75.00%, ele é "falso" e indesejado. O modelo recomenda `Huíla` a *todos*, incluindo ao User 1 que nunca lá foi, e falha em recomendar `Malanje`, `Namibe`, `Bié`, etc.

**Próximo Passo:**
Construir o `03_Content_Based_Model.ipynb`. Neste notebook, vamos usar o ficheiro **`mock_user_features.csv`** (com `pref_cultura`, `pref_praia`, etc.) para criar **recomendações personalizadas**, com o objectivo de aumentar a Precisão e o Recall, e gerar uma diversidade de recomendações muito mais inteligente.